In [14]:
import fitz
import numpy as np
from collections import Counter

In [6]:
pdf_address = "a.pdf"
doc = fitz.open(pdf_address)
page = doc.load_page(13)
page_dict = page.get_text("dict")
page_blocks = page_dict["blocks"]
page_blocks_2 = page.get_text("blocks")
page_text = page.get_text("text")

blocks_bbox_text = []
for i in range(len(page_blocks_2)):
    blocks_bbox_text.append(page_blocks_2[i][0:5])

!!! if a line has multiple spans! This should be considered and sall be added.!!!

In [39]:
page_blocks
sorted_page_blocks = sorted(page_blocks, key=lambda x: (x['bbox'][0], x['bbox'][1]))
sorted_page_blocks[0]['lines'][0]['spans'][0]['text']

{'number': 1,
 'type': 0,
 'bbox': (68.03150177001953,
  34.0159912109375,
  204.282470703125,
  43.0159912109375),
 'lines': [{'spans': [{'size': 9.0,
     'flags': 4,
     'font': 'FrutigerLTCom-Light',
     'color': 0,
     'ascender': 0.75,
     'descender': -0.25,
     'text': 'M A S C H I N E L L E S  L E R N E N',
     'origin': (68.03150177001953, 40.7659912109375),
     'bbox': (68.03150177001953,
      34.0159912109375,
      204.282470703125,
      43.0159912109375)}],
   'wmode': 0,
   'dir': (1.0, 0.0),
   'bbox': (68.03150177001953,
    34.0159912109375,
    204.282470703125,
    43.0159912109375)}]}

In [19]:
page_blocks[0]['bbox'][1]

805.250244140625

In [44]:
def is_close(cluster, line, x_threshold, y_threshold):
    # Check if the line is close to the cluster in both x and y directions
    x_close = abs((cluster['bbox'][0] + cluster['bbox'][2])/2 - (line['bbox'][0] + line['bbox'][2])/2) <= x_threshold
    y_close = abs(cluster['bbox'][3] - line['bbox'][1]) <= y_threshold
    return x_close and y_close

In [45]:
def merge_clusters(cluster, new_line):
    # Merge the new line into the existing cluster
    x1 = min(cluster['bbox'][0], new_line['bbox'][0])
    y1 = min(cluster['bbox'][1], new_line['bbox'][1])
    x2 = max(cluster['bbox'][2], new_line['bbox'][2])
    y2 = max(cluster['bbox'][3], new_line['bbox'][3])
    text = cluster['lines'][0]['spans'][0]['text'] + " " + new_line['lines'][0]['spans'][0]['text']
    return {'bbox': (x1, y1, x2, y2), 'text': text}


In [46]:
def find_text_blocks(page_lines, x_threshold=50, y_threshold=10):
    # Sort the list by y1 coordinate and then by x1 coordinate
    sorted_lines = sorted(page_lines, key=lambda k: (k['bbox'][1], k['bbox'][0]))
    
    clusters = []

    for line in sorted_lines:
        merged = False
        for i in range(len(clusters)):
            if is_close(clusters[i], line, x_threshold, y_threshold):
                # Merge with the cluster
                clusters[i] = merge_clusters(clusters[i], line)
                merged = True
                break
        if not merged:
            # Start a new cluster
            clusters.append(line)

    return clusters

In [47]:
def add_cluster_number(page_lines, clusters):
    for line in page_lines:
        for i, cluster in enumerate(clusters):
            if (line['bbox'][0] >= cluster['bbox'][0] and line['bbox'][1] >= cluster['bbox'][1] 
                and line['bbox'][2] <= cluster['bbox'][2] and line['bbox'][3] <= cluster['bbox'][3]):
                line['cluster_number'] = i
                break
    return page_lines

In [48]:
def sort_lines(page_lines, clusters):
    #sort lines of page based on clusters coordination
    sorted_clusters = sorted(clusters, key=lambda k: (k['bbox'][1], k['bbox'][0]))
    sorted_lines = add_cluster_number(page_lines, sorted_clusters)
    sorted_lines = sorted(sorted_lines, key=lambda k: (k['cluster_number'], k['bbox'][1], k['bbox'][0]))

    return sorted_lines

In [49]:
def find_titles(sorted_lines_blocks):
    #blocks = page.get_text("dict", flags=11)["blocks"]
    blocks = sorted_lines_blocks
    page_font_sizes = []
    page_font_names = []
    for b in blocks:  # iterate through the text blocks
        for l in b["lines"]:  # iterate through the text lines
            for s in l["spans"]:  # iterate through the text spans
                #print("")
                page_font_sizes.append(s["size"])
                page_font_names.append(s["font"])
                
    page_font_sizes_counter = Counter(page_font_sizes)
    most_common_font_size, most_common_count = page_font_sizes_counter.most_common(1)[0]

    if most_common_count / len(page_font_sizes) > 0.5:
        title_font_sizes = [x for x in page_font_sizes if x > most_common_font_size]
    else:
        return None
    
    title_blocks = []
    for b in blocks:  # iterate through the text blocks
        for l in b["lines"]:  # iterate through the text lines
            for s in l["spans"]:  # iterate through the text spans
                if s["size"] in title_font_sizes:
                    title_blocks.append(b)

    return title_blocks

In [59]:
x = sort_lines(page_blocks, find_text_blocks(page_blocks))

KeyError: 'lines'

In [50]:
def is_title(page_block, ptitles):
    if page_block is None:
        return False
    if not ptitles:
        return False
    for item in ptitles:
        if item['bbox'] == page_block['bbox'][:4]:
            #print("bbox of title:", item['bbox'], "\n", "bbox of block:", pblock[:4])
            return True
    return False

In [51]:
def extract_image_from_page(pdf_document, page, page_number):
    image_list = page.get_images(full=True)
    if image_list is None:
        return None
    else:
        images_name = []
        info_names = []   #write decription about info of images in PyMuPDF.
        images_y0 = []
        for img_index, img_info in enumerate(image_list):
            xref = img_info[0]
            image_info_name = img_info[7]
            base_image = pdf_document.extract_image(xref)
            image_bytes = base_image["image"]
            image_filename = f"image_{page_number+1}_{img_index + 1}.png"
            y0 = page.get_image_bbox(img_info)[1]  
            images_name.append(image_filename)
            info_names.append(image_info_name)
            images_y0.append(y0)
            with open(image_filename, "wb") as image_file:
                image_file.write(image_bytes)
        return images_name, info_names, images_y0

In [56]:
def make_master_list(pdf_doc):
    master_list = []

    # Iterate through each page
    for page_num in range(len(pdf_doc)):
        page = pdf_doc.load_page(page_num)

        # Extract text blocks from the page
        #text_blocks = page.get_text("blocks")
        text_lines = page.get_text("dict", flags=11)["blocks"]
        text_blocks = find_text_blocks(text_lines)
        text_blocks = sort_lines(text_lines, text_blocks)

        page_titles = find_titles(text_blocks)

        image_names, image_info_names, images_y0 = extract_image_from_page(pdf_doc, page, page_num)

        # Add page number to each block and append to master list
        for i, block in enumerate(text_blocks):
            # Create a dictionary for the block
            block_dict = {
                "page": page_num,
                "bbox": block['bbox'][0:4],  # Position of the text block
                "text": block['lines'][0]['spans'][0]['text'],   # Text content of the block
                "image": [],
                "title": is_title(block, page_titles)
            }
            
            next_block = text_blocks[i+1] if i < len(text_blocks) - 1 else None
            for j, image_y0 in enumerate(images_y0):
                if is_title(next_block, page_titles) and image_y0 < next_block[1]:
                    block_dict["image"].append(image_names[j])
    
            master_list.append(block_dict)

    return master_list

In [57]:
make_master_list(doc)

IndexError: list index out of range